In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc

import Qubits

EC = 0.24
EJ = 10
T = 0.1
ng = 0

In [2]:
transmonC = Qubits.transmon_charge(30, EC, EJ, ng)#Values given in Ghz*hbar

transmonC.solve()

gamma_arr = transmonC.T_1_gamma()
print(gamma_arr)

print(1/gamma_arr*1e6)


print(1/np.sum(gamma_arr)*1e6)

Charge matrix element: 3.958842286929474
Phi matrix element: 0.1828437737554734
Qubit energies: -7.870907691820001 and -3.744641737875064
Qubit frequency: 4.126265953944937
[ 15.07061104  17.574875   396.33154481]
[66354.31022351 56899.40897352  2523.14006567]
2331.127142257869


In [3]:
15.07061104/3.95884228692947

3.8068227900255565

In [4]:
transmonF = Qubits.transmon_flux(100, EC, EJ, ng)#Values given in Ghz*hbar

transmonF.solve()

gamma_arr_flux = transmonF.T_1_gamma()
print(gamma_arr_flux)

print(1/gamma_arr_flux)
print(1/np.sum(gamma_arr_flux)*1e6)

Charge matrix element: 3.983841409718074
Phi matrix element: 0.23476298453635797
Qubit energies: -7.851195569144888 and -3.690133178043594
Qubit frequency: 4.161062391101295
[ 15.03895608  17.83499931 511.40228693]
[0.06649398 0.05606953 0.00195541]
1837.3023149845899


In [5]:
11.36176931/5.801588912968675

1.9583892413683925

In [6]:
gatemonF = Qubits.gatemon_flux(100, EC, EJ, T, ng)

gatemonF.solve()

gamma_gatemonF = gatemonF.T_1_gamma()

print(gamma_gatemonF)

print(1/gamma_gatemonF)
print(1/np.sum(gamma_gatemonF)*1e6)

Charge matrix element: 0.7500526439697275
Phi matrix element: 0.3154754024759674
Qubit energies: -9.610077287398083 and -8.813597310655274
Qubit frequency: 0.796479976742809
[ 14.79233593   0.6427372  399.18226207]
[0.06760258 1.55584584 0.00250512]
2411.862493692725


In [7]:
gatemonC = Qubits.gatemon_charge(40, EC, EJ, T, ng)

gatemonC.solve()

gamma_gatemonC = gatemonC.T_1_gamma()

print(gamma_gatemonC)

print(1/gamma_gatemonC)
print(1/np.sum(gamma_gatemonC)*1e6)

Charge matrix element: 0.16381114621241938
Phi matrix element: 0.03899681646034193
Qubit energies: -9.61336420636505 and -8.814955862227505
Qubit frequency: 0.7984083441375454
[ 3.22283639  0.14071336 49.36131295]
[0.31028569 7.10664557 0.02025878]
18966.38414365044
